In [1]:
import sys
sys.path.append('../')

In [32]:
import pandas as pd
import numpy as np
# import train_test_split 
from sklearn.model_selection import train_test_split
from os import listdir
from methods import ARTICLES_DIR,remove_stop_words
from typing import List,Dict
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

TODO:
 1. load all the documents
 2. vectorize each document, 1 for word existing in the document, 0 for not existing
 3. cluster the documents

לטעון את כל המסמכים

להפריד כל מסמך לפי רווח רגיל

לעשות אוסף של המילים

ולעבור כל מסמך ומסמך, האם מילה קיימת או לא.

בלי stemming ובלי הורדת מילות טפל

In [3]:
#TODO: change the directories and remove files to get to 50 total articles.

paths=[(ARTICLES_DIR,50),
       (f"{ARTICLES_DIR}/other groups/5 Technology for cultural heritage/text",18),
       (f"{ARTICLES_DIR}/other groups/12 Visitors guide archeological sites/text files",19),
       (f"{ARTICLES_DIR}/other groups/14 Information presentation museum/txt",18)
    ]

In [4]:
for path,_ in paths:
    path=f"../{path}"
    print(len(listdir(path)))

50
52
51
51


In [5]:
# כתוב פונקציה שמקבלת מחרוזת, מחזירה מערך של מחרוזות של המילים במחרוזת המסופקת
# מוסיפה את הרשימה שחוזרת לתוך אובייקט מסוג set

def split_str(document:str)->List[str]:
    return document.split()


In [6]:
dic:Dict[int,Dict[str,bool]]={}
set_of_words=set()

In [7]:
counter=0
# בניית מילון המונחים
for path,count in paths:
    path=f"../{path}"
    arts=listdir(path)
    for i in range(count):
        art=arts[i]
        if art=='.DS_Store': continue
        try:
            with open(f"{path}/{art}",'r',encoding="UTF-8") as file:
                set_of_words.update(split_str(file.read()))
            counter+=1
            
        except:
            print("Error reading file",path,art)
            i-=1
    print(path,counter)


Error reading file .././articels other groups
.././articels 48
.././articels/other groups/5 Technology for cultural heritage/text 65
.././articels/other groups/12 Visitors guide archeological sites/text files 83
.././articels/other groups/14 Information presentation museum/txt 100


In [8]:
counter

100

In [9]:
with open('../stop_words_english.txt','r',encoding="UTF-8") as file:
    stop_words=set(file.read().split())

In [10]:
set_of_stop_words=set(stop_words)

In [11]:
set_of_words-=set_of_stop_words

In [12]:
len(set_of_words)
dic.clear()

In [13]:
# dic:Dict[int,Dict[str,bool]]={}
id=1
for path,count in paths:
    path=f"../{path}"
    arts=listdir(path)
    for i in range(count):
        art=arts[i]
        if art=='.DS_Store': continue
        try:
            with open(f"{path}/{art}",'r',encoding="UTF-8") as file:
                temp:Dict[str,bool]={}
                content=set(remove_stop_words(file.read()).split())
                
                # content=remove_stop_words(file.read())
                for word in set_of_words:
                    temp[word]=1 if word in content else 0
                    # temp[word]=content.count(word)
                dic[id]=temp
                id+=1
        except:
            print("Error reading file",path,art)
            i-=1

Error reading file .././articels other groups


In [14]:
id,len(dic)

(101, 100)

In [15]:
df=pd.DataFrame.from_dict(dic,orient='index')

In [16]:
from pickle import dump


In [17]:
len(df),len(df.columns)

(100, 78977)

In [26]:
df["category"]=df.index.map(lambda x: 1 if int(x//49)==0 else 0)

In [27]:
with open("df after category - model vectors for classification.pcl","wb+") as file:
    dump(df,file)

In [28]:
df.to_csv("df after category - model vectors for classification.csv",index=False)

In [29]:
# df.head()

In [30]:

df["category"].value_counts()
# df.drop(columns=["category"],inplace=True)
#צריכים לראות 50-50

0    52
1    48
Name: category, dtype: int64

In [31]:
# df : collection of vectors
X=df.drop(["category"],axis=1)
y=df["category"]

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.10)

In [ ]:
# אימון classifiers

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier

In [34]:
model_logistic_regression=LogisticRegression()
model_logistic_regression.fit(X_train,y_train)
logistic_regression_pred=model_logistic_regression.predict(X_test)


In [35]:
print(classification_report(y_test,logistic_regression_pred))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.83      1.00      0.91         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10



In [36]:
model_naive_bayes=GaussianNB()
model_naive_bayes.fit(X_train,y_train)
naive_bayes_pred=model_naive_bayes.predict(X_test)

In [37]:
print(classification_report(y_test,naive_bayes_pred))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       1.00      0.40      0.57         5

    accuracy                           0.70        10
   macro avg       0.81      0.70      0.67        10
weighted avg       0.81      0.70      0.67        10



In [44]:
model_k_neighbors_classifier=KNeighborsClassifier(2,weights='distance',algorithm='auto')
model_k_neighbors_classifier.fit(X_train,y_train)
k_neighbors_classifier_pred=model_k_neighbors_classifier.predict(X_test)

In [45]:
print(classification_report(y_test,k_neighbors_classifier_pred,zero_division=0))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         5
           1       0.00      0.00      0.00         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



In [64]:
model_random_forest=RandomForestClassifier(n_estimators=250)
model_random_forest.fit(X_train,y_train)
random_forest_pred=model_random_forest.predict(X_test)

In [65]:
print(classification_report(y_test,random_forest_pred))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.83      1.00      0.91         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10



In [68]:
# df.info()